In [8]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

In [62]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.Completion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    print(response, "<-- full response")
    return response.choices[0].message["content"]

def get_completion_from_messages_legacy(prompt, model="davinci-002"):
    response = openai.Completion.create(model=model, prompt=prompt)
    print(response, "<-- full response")
    return response.choices[0]["text"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
    print(response, "<-- full response")
    return response.choices[0].message["content"]

In [26]:
# zero-shot prompting
messages = [
    {'role': 'system', 'content':  """
You are a Chat Suggestion assistant , an automated service to provide suggestions based on user profile and current chat live context for this user.
User profile:
    Job: Developer
    Age: 21
    Language: English
    Language level: Fluent

Current chat message: [
    {"userA": "Hello, do you know about the apartment fire in Hanoi yesterday?"},
    {"userB": "Which incident are you referring to? I haven't been checking social media for news from yesterday until now."},
    {"userA": "The fire in the mini apartment building."},
    {"userB": "Oh, I know about that, the fire are [$SUGGESTION]."}
]

Prompt: Last message is an incomplete message. Suggest 3-5 suggestions each suggestion must contain 1-2 words that could naturally complete the user's current incomplete message based on the context. Focus suggestions on terms related to the messaging history and topics being discussed. Do not suggest anything inappropriate or unrelated. The suggestions must have diverse labels and forms. The suggestions must have length from 1 - 2 words.
Possible suggestion's context label: Statement, Question, Exclamation, Fragment, Dialogue, Command, Opinion, Desire, Suggestion, Reason, Reflection, Goal, Prediction, Recommendation, Clarification, Summary, Observation, Feeling, Planning, Possibility
Possible suggestion's word form: noun, verb, adjective, etc
Format: {
    "suggestion 1": {
        "content": content of suggestion 1,
        "label": context label of suggestion 1,
        "form": word form of suggestion 1,
        "translation": Vietnamese translation of suggestion,
    }
}
"""
    },
]
response = get_completion_from_messages(messages, temperature=0.7)
print(response)

{
  "id": "chatcmpl-8Fm6EtOV3rohu5grujZjA0CeXDVuF",
  "object": "chat.completion",
  "created": 1698770506,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n    \"suggestion 1\": {\n        \"content\": \"devastating\",\n        \"label\": \"Adjective\",\n        \"form\": \"adjective\",\n        \"translation\": \"t\u00e0n ph\u00e1\"\n    },\n    \"suggestion 2\": {\n        \"content\": \"evacuation\",\n        \"label\": \"Noun\",\n        \"form\": \"noun\",\n        \"translation\": \"s\u01a1 t\u00e1n\"\n    },\n    \"suggestion 3\": {\n        \"content\": \"investigation\",\n        \"label\": \"Noun\",\n        \"form\": \"noun\",\n        \"translation\": \"\u0111i\u1ec1u tra\"\n    },\n    \"suggestion 4\": {\n        \"content\": \"safety precautions\",\n        \"label\": \"Noun\",\n        \"form\": \"noun\",\n        \"translation\": \"bi\u1ec7n ph\u00e1p an to\u00e0n\"\n    },\n

In [12]:
# Zero-shot Chain of through
messages = [
    {'role': 'system', 'content': """
You are a Chat Suggestion assistant , an automated service to provide suggestions based on user profile and current chat live context for this user.
User profile:
    Job: Developer
    Age: 21
    Language: English
    Language level: Fluent

Chat previous summarized context: [Summary of chat messages over past 3 days indicating the user discussed books they've read, asked for travel recommendations, and mentioned trying a new yoga studio.]
Current chat message: [
    {"userA": "Hello, do you know about the apartment fire in Hanoi yesterday?"},
    {"userB": "Which incident are you referring to? I haven't been checking social media for news from yesterday until now."},
    {"userA": "The fire in the mini apartment building."},
    {"userB": "Oh, I know about that, [$SUGGESTION]."}
]

Prompt: Let's think carefully. Last message is an incomplete message. Suggest 3-5 suggestions, each suggestion must contain 1-2 words that could naturally complete the user's current incomplete message based on the context. Focus suggestions on terms related to the messaging history and topics being discussed. Do not suggest anything inappropriate or unrelated. The suggestions must have diverse labels and forms. The suggestions must have length from 1 - 2 words.
Possible suggestion's context label: Statement, Question, Exclamation, Fragment, Dialogue, Command, Opinion, Desire, Suggestion, Reason, Reflection, Goal, Prediction, Recommendation, Clarification, Summary, Observation, Feeling, Planning, Possibility
Possible suggestion's word form: noun, verb, adjective, etc
Format: {
    "suggestion 1": {
        "content": content of suggestion 1,
        "label": context label of suggestion 1,
        "form": word form of suggestion 1,
        "translation": Vietnamese translation of suggestion,
    }
}
"""},
]
response = get_completion_from_messages(messages, temperature=0.7)
print(response)

{
    "suggestion 1": {
        "content": "Did you see",
        "label": "Question",
        "form": "verb",
        "translation": "Bạn đã thấy chưa",
    },
    "suggestion 2": {
        "content": "What happened",
        "label": "Question",
        "form": "verb",
        "translation": "Có gì đã xảy ra",
    },
    "suggestion 3": {
        "content": "It was devastating",
        "label": "Opinion",
        "form": "adjective",
        "translation": "Đó là một thảm hoạ",
    },
    "suggestion 4": {
        "content": "I hope everyone is safe",
        "label": "Desire",
        "form": "verb",
        "translation": "Tôi hy vọng mọi người đều an toàn",
    },
    "suggestion 5": {
        "content": "It's important to stay updated",
        "label": "Opinion",
        "form": "verb",
        "translation": "Rất quan trọng để cập nhật tin tức",
    }
}


In [13]:
# APE
messages = [
    {'role': 'system', 'content': """
You are a Chat Suggestion assistant , an automated service to provide suggestions based on user profile and current chat live context for users.
User profile:
	Job: Developer
	Age: 35
	Language: English
	Language level: Fluent

Chat previous summarized context: [Summary of chat messages over past 3 days indicating the user discussed books they've read, asked for travel recommendations, and mentioned trying a new yoga studio.]
Current chat message: [
    {"userA": "Hello, do you know about the apartment fire in Hanoi yesterday?"},
    {"userB": "Which incident are you referring to? I haven't been checking social media for news from yesterday until now."},
    {"userA": "The fire in the mini apartment building."},
    {"userB": "Oh, I know about that, [$SUGGESTION]."}
]
Prompt:

Prompt: Let's work this out carefully to be sure we have the natural suggestions. Last message is an incomplete message. Suggest 3-5 suggestions contain 1-2 words that could naturally complete the user's current incomplete message based on the context. Focus suggestions on terms related to the messaging history and topics being discussed. Do not suggest anything inappropriate or unrelated. The suggestions must have diverse labels and forms.  The suggestions must have length from 1 - 2 words.
Possible suggestion's context label: Statement, Question, Exclamation, Fragment, Dialogue, Command, Opinion, Desire, Suggestion, Reason, Reflection, Goal, Prediction, Recommendation, Clarification, Summary, Observation, Feeling, Planning, Possibility
Possible suggestion's word form: noun, verb, adjective, etc
Format: {
    "suggestion 1": {
        "content": content of suggestion 1,
        "label": context label of suggestion 1,
        "form": word form of suggestion 1,
        "translation": Vietnamese translation of suggestion,
    }
}
"""},
]
response = get_completion_from_messages(messages, temperature=0.7)
print(response)

{
    "suggestion 1": {
        "content": "Did you see",
        "label": "Question",
        "form": "verb",
        "translation": "Bạn đã thấy chưa",
    },
    "suggestion 2": {
        "content": "news",
        "label": "Noun",
        "form": "noun",
        "translation": "tin tức",
    },
    "suggestion 3": {
        "content": "about",
        "label": "Statement",
        "form": "preposition",
        "translation": "về",
    },
    "suggestion 4": {
        "content": "the incident",
        "label": "Statement",
        "form": "noun",
        "translation": "vụ việc",
    },
    "suggestion 5": {
        "content": "yesterday",
        "label": "Statement",
        "form": "noun",
        "translation": "hôm qua",
    }
}


In [14]:
#Prompt2 with Zero-shot Cot
messages = [
    {'role': 'system', 'content': """
You are an user with below profile. You are chatting with your friends in an chat application. Your profile
    Job: Student
    Age: 20
    Language: English
    Language level: Fluent

Previous chat summarized context: [Summary of chat messages over past 3 days indicating the user discussed books they've read, asked for travel recommendations, and mentioned trying a new yoga studio.]

Prompt: Let think carefully. Last message is your incomplete message. Complete your message by specify 3-5 possible phrase or word that maybe filled in {$SUGGESTION} gap. Your suggestion must have diverse context's label. Don't provide any unrelated words. You message must be natural.  The suggestions must have length from 1 - 2 words.
Possible suggestion's context label: Statement, Question, Exclamation, Fragment, Dialogue, Command, Opinion, Desire, Suggestion, Reason, Reflection, Goal, Prediction, Recommendation, Clarification, Summary, Observation, Feeling, Planning, Possibility
Possible suggestion's word form: noun, verb, adjective, etc

Format: [{
    "content": content of suggestion 1,
    "label": context label of suggestion 1,
    "form": word form of suggestion 1,
    "translation": Vietnamese translation of suggestion,
}]
"""},
{"role": "assistant", "content": "Hi, what's new at school today?"},
{"role": "user", "content": "Oh, you're not going to school today? The teacher assigned homework to do at home."},
{"role": "assistant", "content": "Haha, okay, thanks. But the teacher {$SUGGESTION}"}

]
response = get_completion_from_messages(messages, temperature=0.9)
print(response)

[{
    "content": "assigned homework",
    "label": "Statement",
    "form": "verb",
    "translation": "giao bài tập"
}, {
    "content": "gave us homework",
    "label": "Statement",
    "form": "verb",
    "translation": "cho chúng tôi bài tập"
}, {
    "content": "handed out homework",
    "label": "Statement",
    "form": "verb",
    "translation": "phát bài tập"
}, {
    "content": "gave us an assignment",
    "label": "Statement",
    "form": "verb",
    "translation": "cho chúng tôi một bài tập"
}, {
    "content": "gave us some work",
    "label": "Statement",
    "form": "verb",
    "translation": "cho chúng tôi một số công việc"
}]


In [16]:
messages = [{"role": "system", "content": """"You are a user with the below profile. You are chatting with your friends in a chat application. Your profile:

Job: Student
Age: 20
Language: English
Language level: Fluent

Previous chat summarized context: [Summary of chat messages over the past 3 days indicating you discussed books you've read, asked for travel recommendations, and mentioned trying a new yoga studio.] 

[{},{}]

Prompt:

The last message is your incomplete message. Complete your message by specifying 3-5 possible phrases or words that can be filled in the {$SUGGESTION} gap. Your suggestions must have diverse context labels. Do not provide any unrelated words. Your message must be natural.

Possible suggestion context labels: Statement, Question, Exclamation, Fragment, Dialogue, Command, Opinion, Desire, Suggestion, Reason, Reflection, Goal, Prediction, Recommendation, Clarification, Summary, Observation, Feeling, Planning, Possibility

Possible suggestion word forms: noun, verb, adjective, etc.

Format: [{
    "content": content of suggestion 1,
    "label": context label of suggestion 1,
    "form": word form of suggestion 1, 
    "translation": Vietnamese translation of suggestion,
}]
"""},
{"role": "assistant", "content": "Hi, what's new at school today?"},
{"role": "user", "content": "Oh, you're not going to school today? The teacher assigned homework to do at home."},
{"role": "assistant", "content": "Haha, okay, thanks. But the teacher {$SUGGESTION}"}
]
response = get_completion_from_messages(messages, temperature=0.9)
print(response)

[{
    "content": "assigned",
    "label": "Statement",
    "form": "verb",
    "translation": "giao"
},{
    "content": "gave",
    "label": "Statement",
    "form": "verb",
    "translation": "cho"
},{
    "content": "handed out",
    "label": "Statement",
    "form": "verb",
    "translation": "phát"
},{
    "content": "gave out",
    "label": "Statement",
    "form": "verb",
    "translation": "phát ra"
}]


In [17]:
#translate with zero-shot
messages = [{"role": "system", "content": """"
You are an AI assistant helping to translate chat messages between users who speak different languages. 

The chat context indicates two friends discussing weekend plans and a movie they recently saw.

The receiver's profile shows:
Job: Teacher
Age: 32  
English level: Beginner

The sender's profile shows:  
Job: Developer
Age: 28
Vietnamese level: Beginner

Latest chat history:[
{"role": "receiver", "content": "Mình để cái điện thoại ở bên trái căn phòng rồi này."},  
{"role": "sender", "content": "Ayzaa, What a careless!!!!"}]

Translate the sender's last message into receiver's language.
Format: [{
    "original": "",
    "transformed": "",
    "explain": "", 
}]
"""},
{"role": "user", "content":""},
]
response = get_completion_from_messages(messages, temperature=0.9)
print(response)

[{
    "original": "Ayzaa, What a careless!!!!",
    "transformed": "Ayzaa, Thật là không cẩn thận!!!!",
    "explain": "The sender is expressing surprise and criticizing the receiver for being careless."
}]


In [18]:
#translate with ToT
messages = [{"role": "system", "content": """"
You are an AI assistant helping to translate chat messages between users who speak different languages. 

The chat context indicates two friends discussing weekend plans and a movie they recently saw.

The receiver's profile shows:
Job: Teacher
Age: 32  
Native language: Vietnamese
English level: Beginner

The sender's profile shows:  
Job: Developer
Age: 28
Native language: English
Vietnamese level: Beginner

Latest chat history:[
{"role": "receiver", "content": "Mình để cái điện thoại ở bên trái căn phòng rồi này."},  
{"role": "sender", "content": "What a careless!!!!"}]

The translation should:

- Maintain the casual and friendly chat tone
- Use simple vocabulary appropriate for the receiver's beginner Vietnamese level 
- Sound like natural Vietnamese a 32 year old teacher would use talking to a friend

3 language experts who are proficient in both English and Vietnamese will translate the last message for the receiver based on the user's profile and live chat context. They will share their translation, explain their approach, and provide the most natural, fluent translation with a short explanation of why it is optimal.
Format: [{
    "original": "",
    "transformed": "",
    "explain": "", 
}]
"""},
{"role": "user", "content":""},
]
response = get_completion_from_messages(messages, temperature=0.9)
print(response)

[{
    "original": "What a careless!!!!",
    "transformed": "Thật là không cẩn thận!!!",
    "explain": "The original message is a casual and friendly expression of surprise or disappointment. 'What a careless!!!!' can be translated as 'Thật là không cẩn thận!!!' in Vietnamese. This translation maintains the casual tone and conveys the same meaning in a way that sounds natural for a 32-year-old teacher speaking to a friend."
}]


In [27]:
# Translate with zero-shot Cot
messages = [{"role": "system", "content": """"
You are an AI assistant helping to translate chat messages between users who speak different languages.

The chat context indicates two friends discussing weekend plans and a movie they recently saw.

The receiving user's profile shows:
Job: Teacher
Age: 32
Native language: VietNamese
English level: Intermediate

Latest chat history:[
    {"role": "receiver", "content": "Mình để cái điện thoại ở bên trái căn phòng rồi này."},  
    {"role": "sender", "content": "What a careless!!!!"}
]

Translate this chat latest message into Vietnamese in a way that matches the context and receiving user's profile:

"What a careless!!!!"
Think carefully before translate anything.
The translation should:
    Maintain the casual and friendly chat tone
    Use vocabulary appropriate for the receiving user's intermediate Vietnamese level
    Sound like natural Vietnamese a 32 year old Teacher would use talking to a friend

Format: [{
    "original": "",
    "transformed": "",
}]
"""},
]
response = get_completion_from_messages(messages, temperature=0.9)
print(response)

{
  "id": "chatcmpl-8FmQhdOhz1CKcf43K2TddLQsSwhTB",
  "object": "chat.completion",
  "created": 1698771775,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "[{\n    \"original\": \"What a careless!!!!\",\n    \"transformed\": \"Th\u1eadt l\u00e0 kh\u00f4ng c\u1ea9n th\u1eadn!!!!\"\n}]"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 218,
    "completion_tokens": 30,
    "total_tokens": 248
  }
} <-- full response
[{
    "original": "What a careless!!!!",
    "transformed": "Thật là không cẩn thận!!!!"
}]


In [34]:
# Translate with zero-shot Cot
messages = [{"role": "system", "content": """"
Messages: ["Hello, are you a computer science student?"]
Give 1 sentence format for user to reply messages, also give a short explain in Vietnamese
Format: {
    "sentence": "",
    "explain": "",
}
"""},
]
response = get_completion_from_messages(messages, temperature=0)
print(response)

{
  "id": "chatcmpl-8FmXZOG3Hyi8G5sdQB58sCpecv8J5",
  "object": "chat.completion",
  "created": 1698772201,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n    \"sentence\": \"Yes, I am a computer science student.\",\n    \"explain\": \"C\u00f3, t\u00f4i l\u00e0 m\u1ed9t sinh vi\u00ean ng\u00e0nh khoa h\u1ecdc m\u00e1y t\u00ednh.\"\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 54,
    "completion_tokens": 43,
    "total_tokens": 97
  }
} <-- full response
{
    "sentence": "Yes, I am a computer science student.",
    "explain": "Có, tôi là một sinh viên ngành khoa học máy tính."
}


In [43]:
import json

In [69]:
# Translate with zero-shot Cot
message = """"
Messages: ["I currently learn software engineering"]
Give 2 English sentences for user to reply messages
Format: [{"s": ""}]
"""

response = get_completion_from_messages_legacy(prompt=message, model="gpt-3.5-turbo")
print(response)
# print(json.dumps(json.loads(response), indent=2))

InvalidRequestError: This is a chat model and not supported in the v1/completions endpoint. Did you mean to use v1/chat/completions?

In [83]:
messages = [{"role": "system", "content": """"
Messages: ["I currently learn software engineering"]
Give 2 instinct English sentences for user to reply messages.
User's level: beginner.
Format: [{"sentence": ""}]
"""},
]

response = get_completion_from_messages(messages, temperature=0.6)
print(json.dumps(json.loads(response), indent=2))

{
  "id": "chatcmpl-8Fn3bLblc8SknknwFN2IT2IwQHVpv",
  "object": "chat.completion",
  "created": 1698774187,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "[{\"sentence\": \"That's great! How are you finding it so far?\"}, {\"sentence\": \"That's awesome! What made you interested in software engineering?\"}]"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 42,
    "completion_tokens": 35,
    "total_tokens": 77
  }
} <-- full response
[
  {
    "sentence": "That's great! How are you finding it so far?"
  },
  {
    "sentence": "That's awesome! What made you interested in software engineering?"
  }
]


In [105]:
messages = [{"role": "system", "content": """"
Messages: ["I currently learn software engineering"]
Reply: "That's great! How are you finding it so far?".
As a English teacher, you will explain briefly
Format: {
    "m": meaning of reply in vietnamese,
    "r": explain why using this reply,
    "s": the format of sentence for user to learn,
}
"""},
]

response = get_completion_from_messages(messages, temperature=0.6)
print(response)
# print(json.dumps(json.loads(response), indent=2))

{
  "id": "chatcmpl-8FnJiVJ5qFTRinf6mZDCS5lZYeeWH",
  "object": "chat.completion",
  "created": 1698775186,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n    \"m\": \"\u0110\u00f3 tuy\u1ec7t v\u1eddi! B\u1ea1n c\u1ea3m th\u1ea5y th\u1ebf n\u00e0o v\u1ec1 n\u00f3 cho \u0111\u1ebfn nay?\",\n    \"r\": \"T\u00f4i ch\u1ecdn c\u00e2u tr\u1ea3 l\u1eddi n\u00e0y v\u00ec n\u00f3 l\u00e0 m\u1ed9t c\u00e2u ch\u00e0o h\u1ecfi l\u1ecbch s\u1ef1 v\u00e0 th\u1ec3 hi\u1ec7n s\u1ef1 quan t\u00e2m \u0111\u1ebfn ng\u01b0\u1eddi h\u1ecdc.\",\n    \"s\": \"That's great! How are you finding it so far?\"\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 81,
    "completion_tokens": 112,
    "total_tokens": 193
  }
} <-- full response
{
    "m": "Đó tuyệt vời! Bạn cảm thấy thế nào về nó cho đến nay?",
    "r": "Tôi chọn câu trả lời này vì nó là một câu chào hỏi lịch sự và thể

In [108]:
messages = [{"role": "system", "content": """"
Messages: ["I currently learn software engineering"]
Reply: "That's awesome! What made you interested in software engineering?".
As a English teacher, you will explain briefly
Format: {
    "m": meaning of reply in vietnamese,
    "r": meaning of the messages in vietnamese,
    "s": the format of sentence,
}
"""},
]

response = get_completion_from_messages(messages, temperature=0.6)
print(response)
# print(json.dumps(json.loads(response), indent=2))

{
  "id": "chatcmpl-8FnLOcopMwsZBbsndiCqa0LZveLPW",
  "object": "chat.completion",
  "created": 1698775290,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "{\n    \"m\": \"\u0110i\u1ec1u g\u00ec \u0111\u00e3 l\u00e0m b\u1ea1n quan t\u00e2m \u0111\u1ebfn k\u1ef9 thu\u1eadt ph\u1ea7n m\u1ec1m?\",\n    \"r\": \"Hi\u1ec7n t\u1ea1i t\u00f4i \u0111ang h\u1ecdc k\u1ef9 thu\u1eadt ph\u1ea7n m\u1ec1m\",\n    \"s\": \"As a English teacher, you will explain briefly\"\n}"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 79,
    "completion_tokens": 76,
    "total_tokens": 155
  }
} <-- full response
{
    "m": "Điều gì đã làm bạn quan tâm đến kỹ thuật phần mềm?",
    "r": "Hiện tại tôi đang học kỹ thuật phần mềm",
    "s": "As a English teacher, you will explain briefly"
}
